# Shmoo Plot Demo

In [116]:
# Equipment setup

import pyvisa
import time
import subprocess
import serial

# Establish connection to the power supply
rm = pyvisa.ResourceManager("@py")

# psu = rm.open_resource('TCPIP::169.254.58.10::gpib0,7::INSTR')                  ## for the Agilent power supply

psu = rm.open_resource('TCPIP::169.254.201.77::lan0::INSTR')                    ## for the Keysight power supply

clk = rm.open_resource('TCPIP::169.254.201.101::lan0::INSTR')

# Need to verify that this is the power supply. psu_verification should be 
# something like "Keysight Technologies,E3646A,0,X.X-Y.Y-Z.Z"
psu_verification = psu.query("*IDN?")
clk_verification = clk.query("*IDN?")

print("Power supply:", psu_verification)
print("Clock generator:", clk_verification)

# psu.write("*RST")  # Reset the power supply

start_voltage = 0.5

end_voltage = 1.25

step_size = 0.05  

start_clock = 250

end_clock = 850

step_size_clock = 50

# Loop through the voltage range and set the voltage
current_voltage = start_voltage 

current_clock = start_clock

psu.write("OUTP ON, (@2)")

# Only channel 2 works
# psu.write("INST:SEL OUT2")
psu.write("INST:SEL CH2")

# Set current to 500mA
psu.write("CURR 0.5")

# Start the clock at 20MHz
clk.write(":OUTP1:POS ON") # Initialize clk gen channel 1 output
clk.write(":FUNC:PATT")


# expected_output = "32767 32767 32767 32767 32767 32767 32767 32767 32767 32767"
expected_output = "0 4400 4600 4800 4900 4a00 4b00 4c00 4c80 4d00 4d80 4e00 4e80 4f00 4f80 5000"

Power supply: Keysight Technologies,E36312A,MY57010726,1.0.4-1.0.0-1.04

Clock generator: Agilent Technologies, 81134A, DE42800131, Ver 2.9.3



#### Remember to reset the board!

In [117]:
while current_clock <= end_clock:
     # remake to adjust with the frequency
     subprocess.run(["make", "clean"], cwd="../../")

     with open("../../platform/dsp24/freq.h", "w") as f:
          f.write('''#ifndef __FREQ_H
                    #define __FREQ_H
                    ''')
          f.write(f'''#define MTIME_FREQ     {current_clock}000000
                    #define SYS_CLK_FREQ   {current_clock}000000
                    ''')
          f.write("#endif")

     subprocess.run(["make", "shmoo_dsp"], cwd="../../")

     clk.write(":FREQ 100MHz")
     print(f"**Clock set to: {current_clock}MHz**") 


     while current_voltage <= end_voltage: # Set the voltage   
          psu.write(f"VOLT {current_voltage}")

          voltage = psu.query("VOLT?") 
          print(f"**Voltage set to: {voltage}V**") 

          ratio_integer = hex(int(current_clock / 50))

          # test subprocess with simple hello
          with subprocess.Popen(["uart_tsi", "+tty=/dev/ttyUSB1", "+baudrate=921600", 
                              # "+no_hart0_msip",
                              # "+init_write=0x130000:0x0",
                              # "+init_write=0x140064:0x0",                  ## disable LDO
                              # "+init_write=0x140060:0x0",                  ## disable PLL
                              # "+init_write=0x140074:0x1", 
                              f"+init_write=0x14006C:{ratio_integer}",                  ## ratio_integer
                              # "+init_write=0x140088:0x0",
                              # "+init_write=0x140078:0x1",
                              # "+init_write=0x14007C:0x0",
                              # "+init_write=0x140080:0x1",
                              # "+init_write=0x140084:0x0",
                              # "+init_write=0x140064:0x1",                  ## enable LDO 
                              # "+init_write=0x140060:0x0",                  ## enable PLL
                              "../../build/shmoo_bench.elf"]) as proc:
               measured_voltages = []
               measured_currents = []
               start_time = time.time()
               timeout = False
               with serial.Serial('/dev/ttyUSB4', 115200, timeout=7) as ser:
                    while (ser.inWaiting() == 0):
                         ## measure the voltage + current
                         if time.time() - start_time >= 7:
                              timeout = True
                              break
                         measured_voltages.append(psu.query("MEAS:VOLT?"))
                         measured_currents.append(psu.query("MEAS:CURR?"))
                         # measured_voltages.append(5)
                         # measured_currents.append(5)
                    if timeout:
                         length = min(len(measured_currents), len(measured_voltages))
                         power_consumption = sum([float(measured_voltages[i]) * float(measured_currents[i]) for i in range(length)]) / length
                         with open('../../output.txt', 'a') as file:
                              file.write(f"Freq: {current_clock}, Voltage: {current_voltage}, Output: Invalid, Power: {power_consumption}\n")
                              proc.terminate()
                              proc.wait()     
                    else:
                         line = ser.readline()
                         print(line)
                         new_line = line.decode("utf-8")
     

                         res = "Valid"
                         if not new_line.strip() == expected_output.strip():
                              res = "Invalid"
                         
                         ## calculate power consumption
                         length = min(len(measured_currents), len(measured_voltages))
                         power_consumption = sum([float(measured_voltages[i]) * float(measured_currents[i]) for i in range(length)]) / length
                         with open('../../output.txt', 'a') as file:
                              file.write(f"Freq: {current_clock}, Voltage: {current_voltage}, Output: {res}, Power {power_consumption}\n")
                              proc.terminate()
                              proc.wait()
          


          print("Done: now reset during the sleep before next iteration")
          time.sleep(7)

          current_voltage += step_size
     current_voltage = start_voltage
     current_clock += step_size_clock

print("Finished")

psu.close()

current_clock = start_clock
current_voltage = start_voltage

rm -rf build
cmake -S ./ -B ./build/ -D CMAKE_TOOLCHAIN_FILE=./riscv-gcc.cmake -DCHIP=dsp24
-- The ASM compiler identification is GNU
-- Found assembler: /tools/C/ee290-fa24-2/.conda-env/riscv-tools/bin/riscv64-unknown-elf-gcc
-- The C compiler identification is GNU 13.2.0
-- The CXX compiler identification is GNU 13.2.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /tools/C/ee290-fa24-2/.conda-env/riscv-tools/bin/riscv64-unknown-elf-gcc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /tools/C/ee290-fa24-2/.conda-env/riscv-tools/bin/riscv64-unknown-elf-g++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Including HTIF to target
-- Including CLINT to target
-- ================ Glossy config ================
--  Building for CHIP: "dsp24"
--  Read

/tools/C/tedkim7/sp24-Baremetal-IDE/driver/rocket-chip-blocks/uart/uart.c: In function 'uart_receive':
/tools/C/tedkim7/sp24-Baremetal-IDE/driver/rocket-chip-blocks/uart/uart.c:35:78: warning: unused parameter 'timeout' [-Wunused-parameter]
   35 | Status uart_receive(UART_Type *UARTx, uint8_t *data, uint16_t size, uint32_t timeout) {
      |                                                                     ~~~~~~~~~^~~~~~~
/tools/C/tedkim7/sp24-Baremetal-IDE/driver/rocket-chip-blocks/uart/uart.c: In function 'uart_transmit':
/tools/C/tedkim7/sp24-Baremetal-IDE/driver/rocket-chip-blocks/uart/uart.c:50:85: warning: unused parameter 'timeout' [-Wunused-parameter]
   50 | Status uart_transmit(UART_Type *UARTx, const uint8_t *data, uint16_t size, uint32_t timeout) {
      |                                                                            ~~~~~~~~~^~~~~~~


[  5%] Linking C static library libuart.a
gmake[4]: Leaving directory '/bwrcq/C/tedkim7/sp24-Baremetal-IDE/build'
[  5%] Built target uart
gmake[4]: Entering directory '/bwrcq/C/tedkim7/sp24-Baremetal-IDE/build'
gmake[4]: Leaving directory '/bwrcq/C/tedkim7/sp24-Baremetal-IDE/build'
gmake[4]: Entering directory '/bwrcq/C/tedkim7/sp24-Baremetal-IDE/build'
[  7%] Building C object driver/intel/pll/CMakeFiles/intel-pll.dir/pll.c.obj
[ 10%] Linking C static library libintel-pll.a
gmake[4]: Leaving directory '/bwrcq/C/tedkim7/sp24-Baremetal-IDE/build'
[ 10%] Built target intel-pll
gmake[4]: Entering directory '/bwrcq/C/tedkim7/sp24-Baremetal-IDE/build'
gmake[4]: Leaving directory '/bwrcq/C/tedkim7/sp24-Baremetal-IDE/build'
gmake[4]: Entering directory '/bwrcq/C/tedkim7/sp24-Baremetal-IDE/build'
[ 12%] Building C object driver/rocket-chip/clint/CMakeFiles/clint.dir/clint.c.obj
[ 15%] Linking C static library libclint.a
gmake[4]: Leaving directory '/bwrcq/C/tedkim7/sp24-Baremetal-IDE/build'
[

/tools/C/tedkim7/sp24-Baremetal-IDE/driver/rocket-chip-blocks/i2c/i2c.c: In function 'i2c_init':
/tools/C/tedkim7/sp24-Baremetal-IDE/driver/rocket-chip-blocks/i2c/i2c.c:13:45: warning: unused parameter 'I2C_init' [-Wunused-parameter]
   13 | void i2c_init(I2C_Type *I2Cx, I2C_InitType *I2C_init) {
      |                               ~~~~~~~~~~~~~~^~~~~~~~
/tools/C/tedkim7/sp24-Baremetal-IDE/driver/rocket-chip-blocks/i2c/i2c.c: In function 'i2c_wait_for_flag':
/tools/C/tedkim7/sp24-Baremetal-IDE/driver/rocket-chip-blocks/i2c/i2c.c:30:9: warning: implicit declaration of function 'CLINT_getTime' [-Wimplicit-function-declaration]
   30 |     if (CLINT_getTime() > (timestart + timeout)) {
      |         ^~~~~~~~~~~~~
/tools/C/tedkim7/sp24-Baremetal-IDE/driver/rocket-chip-blocks/i2c/i2c.c:30:25: warning: comparison of integer expressions of different signedness: 'int' and 'uint32_t' {aka 'unsigned int'} [-Wsign-compare]
   30 |     if (CLINT_getTime() > (timestart + timeout)) {
      |    

[ 25%] Linking C static library libi2c.a
gmake[4]: Leaving directory '/bwrcq/C/tedkim7/sp24-Baremetal-IDE/build'
[ 25%] Built target i2c
gmake[4]: Entering directory '/bwrcq/C/tedkim7/sp24-Baremetal-IDE/build'
gmake[4]: Leaving directory '/bwrcq/C/tedkim7/sp24-Baremetal-IDE/build'
gmake[4]: Entering directory '/bwrcq/C/tedkim7/sp24-Baremetal-IDE/build'
[ 28%] Building C object driver/rocket-chip-blocks/spi/CMakeFiles/spi.dir/spi.c.obj


...

b'0 4400 4600 4800 4900 4a00 4b00 4c00 4c80 4d00 4d80 4e00 4e80 4f00 4f80 5000 \r\n'
Starting UART-based TSI
Usage: ./uart_tsi +tty=/dev/pts/xx <PLUSARGS> <bin>
       ./uart_tsi +tty=/dev/ttyxx  <PLUSARGS> <bin>
       ./uart_tsi +tty=/dev/ttyxx  +no_hart0_msip +init_write=0x80000000:0xdeadbeef none
       ./uart_tsi +tty=/dev/ttyxx  +no_hart0_msip +init_read=0x80000000 none
       ./uart_tsi +tty=/dev/ttyxx  +selfcheck <bin>
       ./uart_tsi +tty=/dev/ttyxx  +baudrate=921600 <bin>
Attempting to open TTY at /dev/ttyUSB1
Checking connection status with /dev/ttyUSB1
Connection succeeded
Done, shutting down, flushing UART
Done: now reset during the sleep before next iteration
**Voltage set to: +1.15000000E+00
V**


Writing 14006c with 11
Done writing 14006c with 11


Starting UART-based TSI
Usage: ./uart_tsi +tty=/dev/pts/xx <PLUSARGS> <bin>
       ./uart_tsi +tty=/dev/ttyxx  <PLUSARGS> <bin>
       ./uart_tsi +tty=/dev/ttyxx  +no_hart0_msip +init_write=0x80000000:0xdeadbeef none
       ./uart_tsi +tty=/dev/ttyxx  +no_hart0_msip +init_read=0x80000000 none
       ./uart_tsi +tty=/dev/ttyxx  +selfcheck <bin>
       ./uart_tsi +tty=/dev/ttyxx  +baudrate=921600 <bin>
Attempting to open TTY at /dev/ttyUSB1
Checking connection status with /dev/ttyUSB1
Connection succeeded
Done, shutting down, flushing UART
Done: now reset during the sleep before next iteration
**Voltage set to: +1.20000000E+00
V**


Writing 14006c with 11
Done writing 14006c with 11


b'0 4400 4600 4800 4900 4a00 4b00 4c00 4c80 4d00 4d80 4e00 4e80 4f00 4f80 5000 \r\n'
Starting UART-based TSI
Usage: ./uart_tsi +tty=/dev/pts/xx <PLUSARGS> <bin>
       ./uart_tsi +tty=/dev/ttyxx  <PLUSARGS> <bin>
       ./uart_tsi +tty=/dev/ttyxx  +no_hart0_msip +init_write=0x80000000:0xdeadbeef none
       ./uart_tsi +tty=/dev/ttyxx  +no_hart0_msip +init_read=0x80000000 none
       ./uart_tsi +tty=/dev/ttyxx  +selfcheck <bin>
       ./uart_tsi +tty=/dev/ttyxx  +baudrate=921600 <bin>
Attempting to open TTY at /dev/ttyUSB1
Checking connection status with /dev/ttyUSB1
Connection succeeded
Done, shutting down, flushing UART
Done: now reset during the sleep before next iteration
Finished


#### Generate Shmoo Plot

In [118]:
import matplotlib.pyplot as plt
import numpy as np

def read_data(file_name):
    freq_values = []
    voltage_values = []
    valid_values = []
    power_values = []

    with open(file_name, 'r') as file:
        for line in file:
            parts = line.strip().split(', ')
            freq = float(parts[0].split(': ')[1])
            voltage = round(float(parts[1].split(': ')[1]), 2)
            valid = parts[2].split(': ')[1]
            power = float(parts[3].split(': ')[1])

            freq_values.append(freq)
            voltage_values.append(voltage)
            valid_values.append(valid)
            power_values.append(power)
    
    return freq_values, voltage_values, valid_values, power_values

file_name = '../../output.txt'

freq_values, voltage_values, valid_values, power_values = read_data(file_name)

unique_freqs = np.unique(freq_values)
unique_voltages = np.unique(voltage_values)

shmoo_plot = np.zeros((len(unique_voltages), len(unique_freqs)))
power_plot = np.zeros((len(unique_voltages), len(unique_freqs)))

for i, freq in enumerate(unique_freqs):
    for j, voltage in enumerate(unique_voltages):
        validity = valid_values[i * len(unique_voltages) + j]
        if validity == "Valid":
            shmoo_plot[j, i] = 1
            power_plot[j, i] = power_values[i * len(unique_voltages) + j]

plt.figure(figsize=(12, 8))

cmap = plt.cm.colors.ListedColormap(['red', 'springgreen'])

plt.imshow(shmoo_plot, cmap=cmap, aspect='auto', origin='lower')

for i in range(len(unique_freqs)):
    for j in range(len(unique_voltages)):
        if shmoo_plot[j, i] == 1:  # Valid
            plt.text(i, j, f'{power_plot[j, i]:.3f}W', ha='center', va='center', fontsize=8, color='black')

plt.xlabel('Frequency (MHz)')
plt.ylabel('Voltage (V)')
plt.title('Shmoo Plot with Power Consumption')

plt.xticks(np.arange(len(unique_freqs)), unique_freqs, rotation=45)
plt.yticks(np.arange(len(unique_voltages)), unique_voltages)

plt.tight_layout()
plt.show()


Error closing VISA link: [Errno 104] Connection reset by peer


IndexError: list index out of range